In [ ]:
import os
import json
from FileHandler import ask_directory, check_directory, select_excel_files, confirm_action
from SubTables import process_csv_file
from MasterTable import combine_and_organize

try:
    # Load the modification times from a file (if it exists)
    mod_times_file = 'modification_times.json'
    if os.path.exists(mod_times_file):
        with open(mod_times_file, 'r') as f:
            modification_times = json.load(f)
    else:
        modification_times = {}

    # Prompt user to input directories
    input_directory = ask_directory("Enter the address of Input Directory: ")

    # Check if input_directory exists and is valid
    check_directory(input_directory)

    # Select .csv files to process
    selected_files = select_excel_files(input_directory)

    if not selected_files:
        print("Process cancelled. No files selected.")
    else:
        # Ask user for output directory for organized data
        output_directory = ask_directory("Enter the address of Output Directory for Organized Data: ")
        os.makedirs(output_directory, exist_ok=True)

        # Process selected files
        organized_files = []
        for file in selected_files:
            file_path = os.path.join(input_directory, file)
            last_mod_time = os.path.getmtime(file_path)

            # Check if the file has been modified since last processed
            if file in modification_times and modification_times[file] == last_mod_time:
                print(f"Skipping '{file}' as it has not been modified.")
                continue

            if file_path.endswith('.csv'):
                process_func = process_csv_file
            else:
                print(f"Skipping file '{file}'. Unsupported file format.")
                continue

            # Process the file and save organized data
            excel_file, success = process_func(file_path, output_directory)
            if success:
                print(f"Processed '{file}' and saved to '{excel_file}'.")
                organized_files.append(excel_file)

                # Update the modification time
                modification_times[file] = last_mod_time
            else:
                print(f"Processing failed for '{file}'.")

        # Save the modification times
        with open(mod_times_file, 'w') as f:
            json.dump(modification_times, f)

        # After processing, ask user if they want to add data to master table
        if confirm_action():
            # Ask user for output directory for master table
            master_table_output = ask_directory("Enter the address of Output Directory for Master Table: ")
            os.makedirs(master_table_output, exist_ok=True)

            # Combine organized data into master table
            master_table_file = os.path.join(master_table_output, "master_table.xlsx")
            combine_and_organize(output_directory, master_table_file)
            print(f"Master table updated and saved to '{master_table_file}'.")
        else:
            print("Data processing completed. Data was not added to the master table.")

except PermissionError as pe:
    print(f"Permission error occurred: {pe}. Please check permissions.")
except FileNotFoundError as fnf:
    print(f"File not found error: {fnf}.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
